In [1]:
from google.colab import files
uploaded = files.upload()

Saving WA_Fn-UseC_-Telco-Customer-Churn.csv to WA_Fn-UseC_-Telco-Customer-Churn.csv


In [2]:
import pandas as pd
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
# Drop customerID and handle missing values
df.drop('customerID', axis=1, inplace=True)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df = df.dropna()

# Convert categorical columns to dummy variables
df = pd.get_dummies(df, drop_first=True)

# Separate features and target
X = df.drop('Churn_Yes', axis=1).values
y = df['Churn_Yes'].values


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset, DataLoader

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Dataloaders
train_ds = TensorDataset(X_train, y_train)
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)


In [5]:
import torch.nn as nn
class ChurnModel(nn.Module):
    def __init__(self, n_features):
        super(ChurnModel, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(n_features, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

model = ChurnModel(X_train.shape[1])


In [6]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(20):
    for xb, yb in train_dl:
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 0.5206
Epoch 2, Loss: 0.4633
Epoch 3, Loss: 0.5092
Epoch 4, Loss: 0.2335
Epoch 5, Loss: 0.2952
Epoch 6, Loss: 0.4704
Epoch 7, Loss: 0.4461
Epoch 8, Loss: 0.3976
Epoch 9, Loss: 0.3870
Epoch 10, Loss: 0.3382
Epoch 11, Loss: 0.2118
Epoch 12, Loss: 0.5696
Epoch 13, Loss: 0.4137
Epoch 14, Loss: 0.2598
Epoch 15, Loss: 0.4336
Epoch 16, Loss: 0.5175
Epoch 17, Loss: 0.5066
Epoch 18, Loss: 0.4378
Epoch 19, Loss: 0.4126
Epoch 20, Loss: 0.8311


In [7]:
from sklearn.metrics import accuracy_score, classification_report

with torch.no_grad():
    y_pred = model(X_test)
    y_pred_cls = (y_pred > 0.5).float()
    acc = accuracy_score(y_test, y_pred_cls)
    print(f"Test Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred_cls))


Test Accuracy: 0.7910
              precision    recall  f1-score   support

         0.0       0.84      0.88      0.86      1033
         1.0       0.62      0.54      0.58       374

    accuracy                           0.79      1407
   macro avg       0.73      0.71      0.72      1407
weighted avg       0.78      0.79      0.79      1407

